<a href="https://colab.research.google.com/github/rishi-tha16/DL-assignment2/blob/main/Question1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Question1
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# === STEP 1: Load and preprocess data ===
def load_data(path):
    df = pd.read_csv(path, sep='\t', header=None)
    df.dropna(inplace=True)
    return list(zip(df[1], df[0]))  # Latin, Devanagari

train_pairs = load_data('/content/sample_data/hi.translit.sampled.train.tsv')
val_pairs = load_data('/content/sample_data/hi.translit.sampled.train.tsv')

input_texts = [inp for inp, _ in train_pairs]
target_texts = ['\t' + tgt + '\n' for _, tgt in train_pairs]

input_chars = sorted(set(''.join(input_texts)))
target_chars = sorted(set(''.join(target_texts)))

input_token_index = {char: i for i, char in enumerate(input_chars)}
target_token_index = {char: i for i, char in enumerate(target_chars)}

reverse_input_char_index = {i: char for char, i in input_token_index.items()}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens))
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens))
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens))

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# === STEP 2: Build Model ===
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64, epochs=10, validation_split=0.2)

# === STEP 3: Inference Models ===
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# === STEP 4: Decode Sequence ===
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    decoded_sentence = ''

    for _ in range(max_decoder_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n':
            break

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]

    return decoded_sentence.strip()

# === STEP 5: Interactive Prediction ===
def encode_input_text(input_text):
    encoder_input = np.zeros((1, max_encoder_seq_length, num_encoder_tokens))
    for t, char in enumerate(input_text):
        if char in input_token_index:
            encoder_input[0, t, input_token_index[char]] = 1.
    return encoder_input

while True:
    user_input = input("Enter a Latin word (or type 'exit' to quit): ").strip().lower()
    if user_input == 'exit':
        break
    input_seq = encode_input_text(user_input)
    decoded = decode_sequence(input_seq)
    print(f"Predicted Devanagari: {decoded}")


Epoch 1/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 115s 203ms/step - accuracy: 0.0613 - loss: 1.1892 - val_accuracy: 0.0623 - val_loss: 1.2939
Epoch 2/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 113s 205ms/step - accuracy: 0.0796 - loss: 1.1000 - val_accuracy: 0.0632 - val_loss: 1.3076
Epoch 3/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 111s 200ms/step - accuracy: 0.0914 - loss: 1.0581 - val_accuracy: 0.0581 - val_loss: 1.3276
Epoch 4/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 113s 205ms/step - accuracy: 0.1056 - loss: 1.0174 - val_accuracy: 0.0577 - val_loss: 1.3406
Epoch 5/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 138s 199ms/step - accuracy: 0.1128 - loss: 0.9937 - val_accuracy: 0.0618 - val_loss: 1.3214
Epoch 6/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 145s 205ms/step - accuracy: 0.1171 - loss: 0.9796 - val_accuracy: 0.0605 - val_loss: 1.3078
Epoch 7/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 141s 204ms/step - accuracy: 0.1197 - loss: 0.9687 - val_accuracy: 0.0609 - val_loss: 1.3036
Epoch 8/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 141s 202ms/step - accuracy: 0.1234 -

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load and preprocess data
def load_data(path):
    df = pd.read_csv(path, sep='\t', header=None)
    df.dropna(inplace=True)
    return list(zip(df[1], df[0]))  # Latin, Devanagari

train_pairs = load_data('/content/sample_data/hi.translit.sampled.train.tsv')

input_texts = [inp for inp, _ in train_pairs]
target_texts = ['\t' + tgt + '\n' for _, tgt in train_pairs]

# Build vocab
input_chars = sorted(set(''.join(input_texts)))
target_chars = sorted(set(''.join(target_texts)))
input_token_index = {char: i + 1 for i, char in enumerate(input_chars)}  # start from 1
target_token_index = {char: i + 1 for i, char in enumerate(target_chars)}  # start from 1

reverse_input_char_index = {i: char for char, i in input_token_index.items()}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

# Hyperparameters
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1
max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)
embedding_dim = 128
latent_dim = 256

# Tokenize and pad
def tokenize(texts, token_index, max_len):
    sequences = [[token_index.get(char, 0) for char in text] for text in texts]
    return pad_sequences(sequences, maxlen=max_len, padding='post')

encoder_input_data = tokenize(input_texts, input_token_index, max_encoder_seq_length)
decoder_input_data = tokenize(target_texts, target_token_index, max_decoder_seq_length)
decoder_target_data = np.zeros_like(decoder_input_data)

# Decoder target is shifted version of decoder input
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]

# Build Model
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(num_encoder_tokens, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(
    [encoder_input_data, decoder_input_data],
    np.expand_dims(decoder_target_data, -1),  # required for sparse loss
    batch_size=64,
    epochs=50,
    validation_split=0.2
)

# Inference models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_states2 = [state_h2, state_c2]
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

# Decode function
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['\t']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index.get(sampled_token_index, '')

        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
        else:
            decoded_sentence += sampled_char

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

# Encode function for inference
def encode_input_text(input_text):
    input_seq = [input_token_index.get(char, 0) for char in input_text]
    input_seq = pad_sequences([input_seq], maxlen=max_encoder_seq_length, padding='post')
    return input_seq

# Interactive Testing
while True:
    user_input = input("Enter a Latin word (or type 'exit' to quit): ").strip().lower()
    if user_input == 'exit':
        break
    input_seq = encode_input_text(user_input)
    decoded = decode_sequence(input_seq)
    print(f"Predicted Devanagari: {decoded}")


Epoch 1/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 126s 219ms/step - accuracy: 0.7163 - loss: 1.2097 - val_accuracy: 0.7509 - val_loss: 1.0740
Epoch 2/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 142s 220ms/step - accuracy: 0.7841 - loss: 0.7688 - val_accuracy: 0.7873 - val_loss: 0.9245
Epoch 3/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 144s 223ms/step - accuracy: 0.8542 - loss: 0.4897 - val_accuracy: 0.8296 - val_loss: 0.6900
Epoch 4/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 119s 216ms/step - accuracy: 0.9081 - loss: 0.2966 - val_accuracy: 0.8543 - val_loss: 0.5594
Epoch 5/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 144s 220ms/step - accuracy: 0.9328 - loss: 0.2150 - val_accuracy: 0.8615 - val_loss: 0.5204
Epoch 6/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 140s 217ms/step - accuracy: 0.9458 - loss: 0.1739 - val_accuracy: 0.8626 - val_loss: 0.5380
Epoch 7/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 122s 222ms/step - accuracy: 0.9539 - loss: 0.1467 - val_accuracy: 0.8675 - val_loss: 0.5280
Epoch 8/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 121s 219ms/step - accuracy: 0.9604 -